In [23]:
import numpy as np

In [2]:
data_path = "traffic-report-mar-2023.csv"
df = pd.read_csv(data_path)

In [3]:
df

,From,To,Node,Site ID,Site Name,Description,Av Speed,85th %ile,24 Hour,Total
0,NaN,NaN,NaN,NaN,NaN,NaN,(km/h),(km/h),NaN,NaN
1,2023-03-01,2023-03-31,SAUDI,10012460.0,10012460.0,Al Baha to Makhwa Road,-,-,-,-
2,2023-03-01,2023-03-31,SAUDI,10020150.0,10020150.0,Baljuraishi to Al Baha Road - Transfered,-,-,-,-
3,2023-03-01,2023-03-31,SAUDI,10020151.0,10020151.0,Al Baha to Baljuraishi Road - Transferred,-,-,-,-
4,2023-03-01,2023-03-31,SAUDI,10023400.0,10023400.0,Al Baha to Al Aqiq Road - Road No. 3400 - Tran...,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...
244,2023-03-01,2023-03-31,SAUDI,80700151.0,80700151.0,Najran to Khamis Mushyit Road,-,-,-,-
245,2023-03-01,2023-03-31,SAUDI,80802090.0,80802090.0,Sheaar to Amohaiyl Road,86.6,108,5408,167651
246,2023-03-01,2023-03-31,SAUDI,80902500.0,80902500.0,Bisha to Al Alaia Road,93.9,111,1893,58670
247,2023-03-01,2023-03-31,SAUDI,90316910.0,90316910.0,Kodmi to Harud Road,84.5,103,2717,84217


In [4]:
print("Original DataFrame:")
print(df.head())

Original DataFrame:
         From          To   Node     Site ID   Site Name  \
0         NaN         NaN    NaN         NaN         NaN   
1  2023-03-01  2023-03-31  SAUDI  10012460.0  10012460.0   
2  2023-03-01  2023-03-31  SAUDI  10020150.0  10020150.0   
3  2023-03-01  2023-03-31  SAUDI  10020151.0  10020151.0   
4  2023-03-01  2023-03-31  SAUDI  10023400.0  10023400.0   

                                         Description Av Speed 85th %ile  \
0                                                NaN   (km/h)    (km/h)   
1                             Al Baha to Makhwa Road        -         -   
2           Baljuraishi to Al Baha Road - Transfered        -         -   
3          Al Baha to Baljuraishi Road - Transferred        -         -   
4  Al Baha to Al Aqiq Road - Road No. 3400 - Tran...        -         -   

  24 Hour Total  
0     NaN   NaN  
1       -     -  
2       -     -  
3       -     -  
4       -     -  


In [24]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification

# Define the model name
model_name = 'bert-base-uncased'  # You can choose different sizes, e.g., 'bert-large-uncased'

# Load pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(model_name)

# Optionally, modify the model for your specific task
# For example, if your task involves recommending tourist attractions, you might add a classification layer on top of BERT:
# model.classifier = nn.Linear(in_features=model.config.hidden_size, out_features=num_classes)

# Print model architecture
print(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [9]:
# Concatenate relevant columns to form input text sequences
data['input_text'] = data['From'] + ' ' + data['To'] + ' ' + data['Description']

# Example of incorporating numerical features
# For demonstration purposes, let's concatenate 'Av Speed' with the input text sequences
data['input_text_with_speed'] = data['input_text'] + ' Avg Speed: ' + data['Av Speed'].astype(str)

# Prepare numerical features separately
numerical_features = data[['Av Speed', '85th %ile', '24 Hour', 'Total']]

# Display the first few rows of prepared input data and numerical features
print(data[['input_text', 'input_text_with_speed']].head())
print(numerical_features.head())

                                          input_text  \
0                                                NaN   
1       2023-03-01 2023-03-31 Al Baha to Makhwa Road   
2  2023-03-01 2023-03-31 Baljuraishi to Al Baha R...   
3  2023-03-01 2023-03-31 Al Baha to Baljuraishi R...   
4  2023-03-01 2023-03-31 Al Baha to Al Aqiq Road ...   

                               input_text_with_speed  
0                                                NaN  
1  2023-03-01 2023-03-31 Al Baha to Makhwa Road A...  
2  2023-03-01 2023-03-31 Baljuraishi to Al Baha R...  
3  2023-03-01 2023-03-31 Al Baha to Baljuraishi R...  
4  2023-03-01 2023-03-31 Al Baha to Al Aqiq Road ...  
  Av Speed 85th %ile 24 Hour Total
0   (km/h)    (km/h)     NaN   NaN
1        -         -       -     -
2        -         -       -     -
3        -         -       -     -
4        -         -       -     -


In [37]:
# Check the total number of rows in the DataFrame
print("Total number of rows in the dataset:", len(data))


Total number of rows in the dataset: 249


In [11]:
# Print the first few rows of the DataFrame to check if the columns have been created for all rows
print(data[['input_text', 'input_text_with_speed']].head())

                                          input_text  \
0                                                NaN   
1       2023-03-01 2023-03-31 Al Baha to Makhwa Road   
2  2023-03-01 2023-03-31 Baljuraishi to Al Baha R...   
3  2023-03-01 2023-03-31 Al Baha to Baljuraishi R...   
4  2023-03-01 2023-03-31 Al Baha to Al Aqiq Road ...   

                               input_text_with_speed  
0                                                NaN  
1  2023-03-01 2023-03-31 Al Baha to Makhwa Road A...  
2  2023-03-01 2023-03-31 Baljuraishi to Al Baha R...  
3  2023-03-01 2023-03-31 Al Baha to Baljuraishi R...  
4  2023-03-01 2023-03-31 Al Baha to Al Aqiq Road ...  


In [ ]:
#Here I want to test the model general text of Bert model.

In [28]:
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertConfig, AdamW
from torch.utils.data import Dataset, DataLoader

In [38]:
# Define the BERT-based model architecture
class CustomBERTModel(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(CustomBERTModel, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.last_hidden_state[:, 0, :])  # Using the [CLS] token representation
        return logits

In [30]:
# Custom Dataset class for text generation task
class CustomDataset(Dataset):
    def __init__(self, texts, labels, numerical_features, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.numerical_features = numerical_features
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        numerical_feat = self.numerical_features[idx]

        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)
        numerical_feat = torch.tensor(numerical_feat, dtype=torch.float32)

        return input_ids, attention_mask, numerical_feat, label

In [39]:
# Define hyperparameters
num_classes = 2  # Example: Assuming binary classification for the best route and channel
batch_size = 32
num_epochs = 5
learning_rate = 2e-5
max_length = 128  # Maximum length of input sequences
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
# Instantiate BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained("bert-base-uncased")
config.output_hidden_states = True  # This is required for Qlora
bert_model = BertModel.from_pretrained('bert-base-uncased', config=config)
model = CustomBERTModel(bert_model, num_classes)
model.to(device)

CustomBERTModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [50]:
# Define your model architecture and import necessary libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AdamW

# Define your model architecture
class YourModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(YourModel, self).__init__()
        # Define your model layers here
        
    def forward(self, input_ids, attention_mask, numerical_features):
        # Define the forward pass of your model here
        return output_logits

# Define your model hyperparameters
input_size = 768  # Update this with the appropriate input size based on your model's architecture
output_size = num_classes  # Update this with the number of output classes in your classification task

# Initialize your model
model = YourModel(input_size, output_size)

# Define other hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5

# Define your device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move your model to the appropriate device
model.to(device)

YourModel()

In [55]:
transport_channels = ['Road', 'Rail', 'Air', 'Sea']
transport_modes = ['Car', 'Bus', 'Train', 'Airplane', 'Ship']

# Create label columns for transport channel and mode
data['Transport_Channel'] = ''  # Initialize empty column for transport channel
data['Transport_Mode'] = ''  # Initialize empty column for transport mode

# Label Assignment (Assuming you have additional data or domain knowledge to infer this information)
# For demonstration purposes, let's assume that the transport channel is always 'Road' and the mode of transport is 'Car'
data['Transport_Channel'] = 'Road'
data['Transport_Mode'] = 'Car'

# Split data into input texts, numerical features, and labels
input_texts = data['input_text'].tolist()
numerical_features = data[['Av Speed', '85th %ile', '24 Hour', 'Total']]
labels_transport_channel = data['Transport_Channel']
labels_transport_mode = data['Transport_Mode']

# Split data into train, validation, and test sets
train_texts, test_texts, train_numerical_features, test_numerical_features, train_labels_channel, test_labels_channel, train_labels_mode, test_labels_mode = train_test_split(
    input_texts, numerical_features, labels_transport_channel, labels_transport_mode, test_size=0.2, random_state=42)
train_texts, val_texts, train_numerical_features, val_numerical_features, train_labels_channel, val_labels_channel, train_labels_mode, val_labels_mode = train_test_split(
    train_texts, train_numerical_features, train_labels_channel, train_labels_mode, test_size=0.2, random_state=42)

# Prepare train, validation, and test datasets
train_dataset = CustomDataset(train_texts, train_labels_channel, train_labels_mode, train_numerical_features, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels_channel, val_labels_mode, val_numerical_features, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels_channel, test_labels_mode, test_numerical_features, tokenizer, max_length)

# Define data loaders for train, validation, and test sets
batch_size = 32  # Update batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


TypeError: CustomDataset.__init__() takes 6 positional arguments but 7 were given

In [57]:

# Create data loaders for train, validation, and test sets
train_dataset = CustomDataset(train_texts, train_labels_channel, train_labels_mode, tokenizer, max_length=128)
val_dataset = CustomDataset(val_texts, val_labels_channel, val_labels_mode, tokenizer, max_length=128)
test_dataset = CustomDataset(test_df['input_text'], test_df['Transport_Channel'], test_df['Transport_Mode'], tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for input_ids, attention_mask, labels_channel, labels_mode in train_loader:
        input_ids, attention_mask, labels_channel, labels_mode = input_ids.to(device), attention_mask.to(device), labels_channel.to(device), labels_mode.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels_channel)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    val_losses = []
    val_preds_channel, val_preds_mode = [], []
    val_labels_channel, val_labels_mode = [], []
    with torch.no_grad():
        for input_ids, attention_mask, labels_channel, labels_mode in val_loader:
            input_ids, attention_mask, labels_channel, labels_mode = input_ids.to(device), attention_mask.to(device), labels_channel.to(device), labels_mode.to(device)
            outputs = model(input_ids, attention_mask)
            val_loss = criterion(outputs, labels_channel)
            val_losses.append(val_loss.item())
            val_preds_channel.extend(torch.argmax(outputs, axis=1).cpu().numpy())
            val_preds_mode.extend(torch.argmax(outputs, axis=1).cpu().numpy())
            val_labels_channel.extend(labels_channel.cpu().numpy())
            val_labels_mode.extend(labels_mode.cpu().numpy())

    # Calculate metrics
    val_accuracy_channel = accuracy_score(val_labels_channel, val_preds_channel)
    val_accuracy_mode = accuracy_score(val_labels_mode, val_preds_mode)
    val_precision_channel = precision_score(val_labels_channel, val_preds_channel, average='macro')
    val_precision_mode = precision_score(val_labels_mode, val_preds_mode, average='macro')
    val_recall_channel = recall_score(val_labels_channel, val_preds_channel, average='macro')
    val_recall_mode = recall_score(val_labels_mode, val_preds_mode, average='macro')
    val_f1_channel = f1_score(val_labels_channel, val_preds_channel, average='macro')
    val_f1_mode = f1_score(val_labels_mode, val_preds_mode, average='macro')

    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {total_loss/len(train_loader):.4f}, '
          f'Val Loss: {np.mean(val_losses):.4f}, '
          f'Val Acc (Channel): {val_accuracy_channel:.4f}, '
          f'Val Acc (Mode): {val_accuracy_mode:.4f}')


NameError: name 'test_df' is not defined